## Simple architecture trained with any dataset
You can learn more about these datasets at:
* CIFAR-10 and CIFAR-100: https://www.cs.toronto.edu/~kriz/cifar.html .
* MNIST: http://yann.lecun.com/exdb/mnist/ .
* FASHION MNIST: https://github.com/zalandoresearch/fashion-mnist .

This example is part of the [K-CAI Neural API](https://github.com/joaopauloschuler/k-neural-api).

In [0]:
#@title Global Settings
from tensorflow import keras
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.datasets
from tensorflow.keras import regularizers
dataset=tensorflow.keras.datasets.fashion_mnist #@param ["tensorflow.keras.datasets.cifar10", "tensorflow.keras.datasets.cifar100", "tensorflow.keras.datasets.mnist", "tensorflow.keras.datasets.fashion_mnist"] {type:"raw"} 
batch_size=64 # @param [32, 64, 128, 256, 512] {type:"raw"} 
epochs=32 # @param [32, 64, 128, 256, 512] {type:"raw"}
l2_decay = 0.000001 #@param {type:"number"}
initial_learning_rate = 0.01 #@param {type:"number"}
verbose=True #@param {type:"boolean"}
bipolar_input=True #@param {type:"boolean"}

if dataset is keras.datasets.cifar10 or dataset is keras.datasets.cifar100:
  input_shape = (32, 32, 3)
else:
  input_shape = (28, 28, 1)


In [0]:
import os
if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull
!cd k && pip install .

In [0]:
import cai.densenet
import cai.util

save_dir = os.path.join(os.getcwd(), 'saved_models')
cai.util.create_folder_if_required(save_dir)
base_model_name = os.path.join(save_dir, 'simple_model_on_any_dataset')

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (5, 5), padding='valid',
                 input_shape=input_shape, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(4, 4)))
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(32, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
if (dataset==keras.datasets.cifar100):
  model.add(keras.layers.Dense(100))
else:
  model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation('softmax'))

if (verbose):
    model.summary()

In [0]:
# Learning rate schedule.
def lrscheduler(epoch):
  return initial_learning_rate * (0.99**epoch)

fit_result,  model_name,  csv_name = cai.datasets.train_model_on_dataset(model, dataset, base_model_name, 
  plrscheduler=lrscheduler, batch_size=batch_size, epochs=epochs, lab=False, verbose=verbose, bipolar=bipolar_input)

In [0]:
model.load_weights(model_name)

In [0]:
from google.colab import files

In [0]:
files.download(csv_name)

In [0]:
files.download(model_name)